In [1]:
# medir tiempos
%load_ext autotime

time: 1.26 ms (started: 2021-06-13 20:23:03 -05:00)


In [2]:
import findspark
findspark.init("/usr/local/spark/spark-3.1.1-bin-hadoop2.7")    #para linux
#findspark.init()                                                 #para windows

time: 25.8 ms (started: 2021-06-13 20:23:03 -05:00)


In [3]:
from pyspark import SparkConf, SparkContext
# Variable de configuración
conf = SparkConf().setMaster("local[*]").setAppName("GBTRegressor")  \
    .set("spark.driver.maxResultSize","0") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "16g") \
    .set("spark.cores.max", "4")

# iniciamos un contexto spark (solo se ejecuta uno. Para ejecutar otra vez , reiniciar el kernel)
sc = SparkContext(conf = conf)
sc

<SparkContext master=local[*] appName=GBTRegressor>

time: 4.27 s (started: 2021-06-13 20:23:06 -05:00)


In [4]:
from pyspark.sql.types import StringType
from pyspark import SQLContext
# le pasamos el contexto anterior
sqlContext = SQLContext(sc)
import os
path="file:"+os.getcwd()+"/Output"
dfspark = sqlContext.read.format('csv').option("header","true").option("inferSchema","true").load(path+'/*.csv')

time: 17.9 s (started: 2021-06-13 20:23:22 -05:00)


In [5]:
sc.getConf().getAll()

[('spark.executor.memory', '4g'),
 ('spark.app.id', 'local-1623633789764'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.cores.max', '4'),
 ('spark.app.startTime', '1623633788537'),
 ('spark.driver.port', '40249'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.maxResultSize', '0'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.name', 'GBTRegressor'),
 ('spark.submit.deployMode', 'client'),
 ('spark.sql.warehouse.dir',
  'file:/home/renzoqamao/my_project_dir/my_project_env/2021-I/InteligenciaArtificial/PracticasCalificadas/PracticaCalificada2-CC421/IA-PracticaCalificada2/spark-warehouse'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', '192.168.1.5')]

time: 57.9 ms (started: 2021-06-13 20:31:04 -05:00)


In [6]:
dfspark.printSchema()

root
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- dif_latitude: double (nullable = true)
 |-- dif_longitude: double (nullable = true)
 |-- distancia: double (nullable = true)
 |-- dia_semana: integer (nullable = true)
 |-- hora: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- anio: integer (nullable = true)

time: 59 ms (started: 2021-06-13 20:31:05 -05:00)


##### creando un vector de caracteristicas

In [7]:
from pyspark.ml.feature import VectorAssembler
ignore=['fare_amount',
        'pickup_datetime',
        'pickup_longitude',
       'pickup_latitude',
       'dropoff_longitude',
       'dropoff_latitude',
       'dif_latitude',
       'dif_longitude']
vectorAssembler = VectorAssembler(inputCols=[x for x in dfspark.columns  
                  if x not in ignore], outputCol = 'features')
df = vectorAssembler.transform(dfspark)
dfmodel = df.select(['features', 'fare_amount'])

time: 327 ms (started: 2021-06-13 20:31:08 -05:00)


In [9]:
# Separar la data en 80% entrenamiento y 20% test
dftrain, dftest = dfmodel.randomSplit([0.8, 0.2], seed=0)
print("tenemos %d datos de entrenamiento y %d datos de prueba." % (dftrain.count(), dftest.count()))

tenemos 1774098 datos de entrenamiento y 444226 datos de prueba.
time: 15.1 s (started: 2021-06-13 20:31:34 -05:00)


### GBTRegressor

In [39]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

gbt = GBTRegressor(labelCol="fare_amount", featuresCol="features", seed=0, lossType="squared")

time: 31.2 ms (started: 2021-06-13 23:51:14 -05:00)


In [40]:
gbtrparamGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [8])
             .addGrid(gbt.maxBins, [20])
             .addGrid(gbt.maxIter, [10])
             .build())

time: 5.95 ms (started: 2021-06-13 23:51:14 -05:00)


In [41]:
gbevaluator = RegressionEvaluator(predictionCol="prediction", labelCol="fare_amount", metricName="rmse")

time: 12.7 ms (started: 2021-06-13 23:51:16 -05:00)


In [42]:
# Create 3-fold CrossValidator
gbcv = CrossValidator(estimator = gbt,
                      estimatorParamMaps = gbtrparamGrid,
                      evaluator = gbevaluator,
                      numFolds = 3)

time: 9.95 ms (started: 2021-06-13 23:51:16 -05:00)


In [43]:
gbcvModel = gbcv.fit(dftrain)

time: 3min 59s (started: 2021-06-13 23:51:17 -05:00)


In [44]:
print(gbcvModel)

CrossValidatorModel_ed43a5db428a
time: 22.9 ms (started: 2021-06-13 23:55:17 -05:00)


In [45]:
gbpredictions = gbcvModel.transform(dftest)

time: 543 ms (started: 2021-06-13 23:55:17 -05:00)


In [46]:
print('RMSE:', gbevaluator.evaluate(gbpredictions))

RMSE: 5.012730518860586
time: 9.21 s (started: 2021-06-13 23:55:17 -05:00)


In [38]:
path="file:"+os.getcwd()+"/models-rmse-4.8/"
gbt.save(path+"gbtregressor")
gbcv.save(path+"gbcrosvalidator")
gbcvModel.save(path+"Model")
gbevaluator.save(path+"evaluator")

time: 3.26 s (started: 2021-06-13 23:38:40 -05:00)


In [47]:
sc.stop()

time: 1.43 s (started: 2021-06-13 23:56:28 -05:00)
